Korišćen dataset [Tennis Major Tournament Match Statistics] (http://archive.ics.uci.edu/ml/datasets/Tennis+Major+Tournament+Match+Statistics?fbclid=IwAR0iXnL0Dkk1J5HxlFvhqWJn_tPJ02eHcYWCHmf5KHjdeFJbCS-7yKjpCLU) predstavlja kolekciju od 8 data setova, po dva seta podataka za svaki grend slem turnir, gde jedan predstavlja podatke za musku konkurenciju, a drugi podatke za zensku konkurenciju. Svaki dataset ima najmanje 76 reda, svaki sa po 42 kolone. Postoje 127 instanci sa 42 atributa. Potrebno je naci prosecne vrednosti statistika potrebne za pobedu u jednom teniskom mecu. 

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics

In [5]:
dataset = pd.read_csv('AusOpen-men-2013.csv')
dataset

,Player1,Player2,Round,Result,FNL1,FNL2,FSP.1,FSW.1,SSP.1,SSW.1,...,BPC.2,BPW.2,NPA.2,NPW.2,TPW.2,ST1.2,ST2.2,ST3.2,ST4.2,ST5.2
0,Lukas Lacko,Novak Djokovic,1,0,0,3,61,35,39,18,...,4,8,8.0,9.0,101,6,7.0,6.0,NaN,NaN
1,Leonardo Mayer,Albert Montanes,1,1,3,0,61,31,39,13,...,0,0,NaN,NaN,42,1,3.0,1.0,NaN,NaN
2,Marcos Baghdatis,Denis Istomin,1,0,0,3,52,53,48,20,...,4,13,12.0,16.0,126,6,7.0,6.0,NaN,NaN
3,Dmitry Tursunov,Michael Russell,1,1,3,0,53,39,47,24,...,1,7,NaN,NaN,79,2,2.0,3.0,NaN,NaN
4,Juan Monaco,Ernests Gulbis,1,0,1,3,76,63,24,12,...,3,5,16.0,28.0,127,1,6.0,7.0,6.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,Andy Murray,Roger Federer,5,0,1,3,61,60,39,28,...,4,17,49.0,66.0,147,6,6.0,6.0,6.0,NaN
122,Rafael Nadal,Grigor Dimitrov,5,1,3,1,73,66,27,22,...,3,6,28.0,41.0,132,6,6.0,6.0,2.0,NaN
123,Tomas Berdych,Stanislas Wawrinka,6,0,1,3,62,71,38,30,...,1,4,14.0,18.0,143,6,6.0,7.0,7.0,NaN
124,Rafael Nadal,Roger Federer,6,1,3,0,65,41,35,22,...,1,2,23.0,42.0,86,6,3.0,3.0,NaN,NaN
